In [1]:
import numpy as np
import pandas as pd


n_steps = 5  # Number of previous games to consider

# Dictionaries to store sequences for each team
team_sequences = {}
df = pd.read_csv('tttt.csv', low_memory=False)

In [2]:
# List of home and away rolling average feature columns
home_features = [col for col in df.columns if col.startswith('home')]
away_features = [col for col in df.columns if col.startswith('away')]

# Ensure that the features are in the same order for home and away
home_features.sort()
away_features.sort()


In [3]:
# Create a mapping from away features to home features
feature_mapping = dict(zip(away_features, home_features))


In [4]:
# Initialize lists for model input and output
X_home = []
X_away = []
y = []

# Iterate over each match in chronological order
for idx, row in df.iterrows():
    match_id = idx  # Assuming the DataFrame index corresponds to the match order
    home_id = row['homeId']
    away_id = row['awayId']
    match_result = row['FTR']  # Encoded as 0: Loss, 1: Draw, 2: Win from the home team's perspective

    # Prepare the home team data
    home_team_seq = team_sequences.get(home_id, [])
    home_features_values = row[home_features].values
    home_team_seq.append(home_features_values)
    team_sequences[home_id] = home_team_seq[-n_steps:]  # Keep only the last n_steps

    # Prepare the away team data (map away features to home features)
    away_team_seq = team_sequences.get(away_id, [])
    away_features_values = row[away_features].rename(feature_mapping).values
    away_team_seq.append(away_features_values)
    team_sequences[away_id] = away_team_seq[-n_steps:]  # Keep only the last n_steps

    # Check if both teams have enough data
    if len(team_sequences[home_id]) == n_steps and len(team_sequences[away_id]) == n_steps:
        # Stack the sequences for both teams
        X_home.append(np.array(team_sequences[home_id]))
        X_away.append(np.array(team_sequences[away_id]))
        y.append(match_result)


In [5]:
X_home = np.array(X_home)  # Shape: (num_samples, n_steps, num_features)
X_away = np.array(X_away)  # Same shape as X_home
y = np.array(y)            # Shape: (num_samples,)


In [6]:
from tensorflow.keras.utils import to_categorical

y = y.astype(int)
y = to_categorical(y, num_classes=3)



IndexError: index 3 is out of bounds for axis 1 with size 3

In [27]:
import numpy as np
import tensorflow as tf

print("NumPy version:", np.__version__)
print("TensorFlow version:", tf.__version__)


ImportError: cannot import name 'typing' from 'numpy' (c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\__init__.py)